In [1]:
import torch
import torch.nn.functional as F
from torch.optim import SGD

from nni_assets.compression.mnist_model import TorchModel, trainer, evaluator, device

# define the model
model = TorchModel().to(device)

# show the model structure, note that pruner will wrap the model layer.
print(model)

C:\Users\adhocmaster\anaconda3\envs\compression\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Failed to download (trying next):
HTTP Error 403: Forbidden



9913344it [00:00, 10595647.69it/s]                                                                                     


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



29696it [00:00, 468197.27it/s]                                                                                         


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



1649664it [00:00, 1846247.36it/s]                                                                                      


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



5120it [00:00, ?it/s]                                                                                                  

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

TorchModel(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu1): ReLU()
  (relu2): ReLU()
  (relu3): ReLU()
  (relu4): ReLU()
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
)


In [2]:
# define the optimizer and criterion for pre-training

optimizer = SGD(model.parameters(), 1e-2)
criterion = F.nll_loss

# pre-train and evaluate the model on MNIST dataset
for epoch in range(3):
    trainer(model, optimizer, criterion)
    evaluator(model)

Average test loss: 0.5685, Accuracy: 8214/10000 (82%)
Average test loss: 0.2903, Accuracy: 9083/10000 (91%)
Average test loss: 0.2022, Accuracy: 9406/10000 (94%)


In [6]:
total = 0
for name, param in model.state_dict().items():
    print(f"{name}: {param.numel()}")
    total += param.numel()
print(f"total: {total}")

conv1.weight: 150
conv1.bias: 6
conv2.weight: 2400
conv2.bias: 16
fc1.weight: 30720
fc1.bias: 120
fc2.weight: 10080
fc2.bias: 84
fc3.weight: 840
fc3.bias: 10
total: 44426


In [ ]:
# Prune configs
config_list = [{
    'op_types': ['Linear', 'Conv2d'], # types of layers to prune
    'exclude_op_names': ['fc3'], # exclude specific layers
    'sparse_ratio': 0.3 # mask 30% of the parameters
}]

In [7]:
from nni.compression.pruning import L1NormPruner
pruner = L1NormPruner(model, config_list)

# show the wrapped model structure, `PrunerModuleWrapper` have wrapped the layers that configured in the config_list.
print(model)

ImportError: cannot import name 'Literal' from 'typing' (C:\Users\adhocmaster\anaconda3\envs\compression\lib\typing.py)